# Project 3: 'Image Analysis'
## First Year Project  
### ITU, Spring 2021

This notebook contains all the code developed to explore, wrangle and analyse the raw data sets for our project, 'Image Analysis'.

Contributors:  
- Andy Bao Nguyen (anbn)
- Florian Micliuc (flmi)
- Mattias Wohlert 
- Sofia Elena Terenziani (sote)

Created: 06-04-2021 

Last modified:

### Imports

In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
from skimage import morphology
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [8]:
#Example_ground_truth
#Features
ex_ground = pd.read_csv("../data/raw/example_ground_truth.csv")
features = pd.read_csv("../data/features/features.csv")

In [10]:
# Create a DataFrame object
df_exground = pd.DataFrame(ex_ground, columns =['image_id', 'melanoma', 'seborrheic_keratosis'])

# Iterate over the index range from
# 0 to max number of columns in dataframe
for i in range(df_exground.shape[1]):

    print('Column Number : ', i)

    # Select column by index position using iloc[]
    columnSeriesObj = df_exground.iloc[:, i]
    print('Column Contents : ', columnSeriesObj.values)

Column Number :  0
Column Contents :  ['ISIC_0001769' 'ISIC_0001852' 'ISIC_0001871' 'ISIC_0003462'
 'ISIC_0003539' 'ISIC_0003582' 'ISIC_0003657' 'ISIC_0003805'
 'ISIC_0004337' 'ISIC_0006651' 'ISIC_0006671' 'ISIC_0006815'
 'ISIC_0006914' 'ISIC_0007141' 'ISIC_0007156' 'ISIC_0007235'
 'ISIC_0007241' 'ISIC_0007332' 'ISIC_0007344' 'ISIC_0007528'
 'ISIC_0007796' 'ISIC_0008025' 'ISIC_0008524' 'ISIC_0009995'
 'ISIC_0010459' 'ISIC_0012099' 'ISIC_0012109' 'ISIC_0012126'
 'ISIC_0012127' 'ISIC_0012143' 'ISIC_0012151' 'ISIC_0012159'
 'ISIC_0012160' 'ISIC_0012191' 'ISIC_0012201' 'ISIC_0012204'
 'ISIC_0012206' 'ISIC_0012210' 'ISIC_0012221' 'ISIC_0012222'
 'ISIC_0012254' 'ISIC_0012256' 'ISIC_0012288' 'ISIC_0012306'
 'ISIC_0012313' 'ISIC_0012316' 'ISIC_0012335' 'ISIC_0012380'
 'ISIC_0012383' 'ISIC_0012400' 'ISIC_0012417' 'ISIC_0012434'
 'ISIC_0012492' 'ISIC_0012513' 'ISIC_0012538' 'ISIC_0012547'
 'ISIC_0012660' 'ISIC_0012684' 'ISIC_0012720' 'ISIC_0012746'
 'ISIC_0012876' 'ISIC_0012927' 'ISIC_0012956' '

In [ ]:
features

# Loading of data and scripts

In [ ]:
%run ../Scripts/fyp2021p3_group00_functions.py

In [ ]:
im = plt.imread("../Data/Raw/example_image/ISIC_0014310.jpg")
plt.imshow(im)

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989 * r * 0.5870 * g + 0.1140 * b
    return gray

grey = rgb2gray(im)
plt.imshow(grey,  cmap = "gray")

# Lesion is darker, and extract a mask of it

In [ ]:
plt.hist(grey)
# Will run for a minute or two

In [ ]:
# Our lesion is in the smaller clusters (The darker pixels), and the greater ones must be the pixels of the skin
im_lim = grey < 4500
plt.imshow(im_lim, cmap = "gray") # We see a little noice within our lesion